In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/train_landscape_images.zip'

Streaming output truncated to the last 5000 lines.
  inflating: landscape_images/3642.jpg  
  inflating: __MACOSX/landscape_images/._3642.jpg  
  inflating: landscape_images/1055.jpg  
  inflating: __MACOSX/landscape_images/._1055.jpg  
  inflating: landscape_images/3905.jpg  
  inflating: __MACOSX/landscape_images/._3905.jpg  
  inflating: landscape_images/338.jpg  
  inflating: __MACOSX/landscape_images/._338.jpg  
  inflating: landscape_images/1874.jpg  
  inflating: __MACOSX/landscape_images/._1874.jpg  
  inflating: landscape_images/1860.jpg  
  inflating: __MACOSX/landscape_images/._1860.jpg  
  inflating: landscape_images/3911.jpg  
  inflating: __MACOSX/landscape_images/._3911.jpg  
  inflating: landscape_images/3939.jpg  
  inflating: __MACOSX/landscape_images/._3939.jpg  
  inflating: landscape_images/462.jpg  
  inflating: __MACOSX/landscape_images/._462.jpg  
  inflating: landscape_images/1690.jpg  
  inflating: __MACOSX/landscape_images/._1690.jpg  
  inflating: landscape_

In [3]:
!python '/content/train_val_split.py'

In [4]:
from typing import Tuple
from torch.utils.data import Dataset
import torchvision.transforms as T
import torch
import numpy as np
import os
from PIL import Image
import io
import glob
from PIL import Image
from skimage import io
import os, os.path
import imageio
from torchvision.transforms import ToTensor
import numpy
import cv2

from torchvision.transforms.functional import resize

class ColorizeData(Dataset):
    def __init__(self,path):
        self.path1 = path
        
        # Initialize dataset, you may use a second dataset for validation if require
        # print('init m eagya')
        # print(len(path))
        # Use the input transform to convert images to grayscale
        self.input_transform = T.Compose([T.ToTensor(),
                                          T.Resize(size=(256,256)),
                                          T.Grayscale(),
                                          T.Normalize((0.5), (0.5))
                                          ])
        # Use this on target images(colorful ones)
        self.target_transform = T.Compose([T.ToTensor(),
                                           T.Resize(size=(256,256)),
                                           T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    def __len__(self) -> int:
        # print("check2")
        # print(len(self.path1))
        return len(self.path1) # return Length of datasetreturn len(self)
        
        pass
    
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor]:
        # print('chec3k')
        # print(index)
        # if torch.is_tensor(index):
        #   index = index.tolist()
        image = cv2.imread(self.path1[index])
        # print(image.shape) #3channel
        self.inputt = self.input_transform(image) #bnw
        # print("COLORIZEDTATAAA SHAPE")
        # print(self.inputt.shape)
        self.targett = self.target_transform(image) #colored
        # print('targt')
        # print(self.targett.shape)
        # print('check')
        return self.inputt, self.targett # Return the input tensor and output tensor for training
        pass

In [5]:
import torch.nn as nn
import torchvision.models as models


class Net(nn.Module):
  def __init__(self, input_size=256):
    super(Net, self).__init__()
    

    # ResNet - First layer accepts grayscale images, 
    # and we take only the first few layers of ResNet for this task
    resnet = models.resnet18(num_classes=100)
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])
    RESNET_FEATURE_SIZE = 128
    ## Upsampling Network
    self.upsample = nn.Sequential(     
      nn.Conv2d(RESNET_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):
    midlevel_features = self.midlevel_resnet(input)
    output = self.upsample(midlevel_features)
    return output


In [6]:
# wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwimport torch
import os

# from colorize_data import ColorizeData
# from basic_model import Net 
import torch.nn as nn

class Trainer:
    def __init__(self):
        pass
        # Define hparams here or load them from a config file
        self.lr = 0.001
        self.model = Net()
        self.epochs = 30
        self.batch_size = 32
        self.criterion = nn.MSELoss()
        #optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=0)
        #model = Net()
        # print('hi1')

    def train(self):
        # print('sdsdf')
        pass

        
        star = os.path.join('/content/images/train', "*.jpg")
        star = glob.glob(star)
        
        # print(star[0].shape)
        # print(len(star)) #3862

                
        star1 = os.path.join('/content/images/val', "*.jpg")
        star1 = glob.glob(star1)
        # print(len(star1)) #3862

            # train_dataset = ColorizeData('/content/images/train') #dataloarders
        train_dataset = ColorizeData(star)
        # print(train_dataset)
        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

        # print(train_dataloader)
        # val_dataset = ColorizeData(star1)
        # val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True)

        
        # print('hi2')
        # Model
        # model = Net()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model=self.model.to(device)
        #model.train()
        # Just right before the actual usage
        
        # model = model.to(device)

        # Loss function to use
        criterion = nn.MSELoss()



        # You may also use a combination of more than one loss function 
        # or create your own.
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001, weight_decay=0.0)  #Momentum=0.9
        # scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9, last_epoch=-1, verbose=True)    
            

            
        # train loop
        for epoch in range(self.epochs):
          self.model.train()
          # print("model enter")
          for input_transform,target_transform in train_dataloader:
                      inputs = torch.tensor(input_transform).to(device)
                      target = torch.tensor(target_transform).to(device)
                      inputs,target = inputs.cuda(),target.cuda()
                      # print("CHECKKKK")
                      # print(target.shape)

                      output_img = self.model(inputs)
                      # print(output_img.shape)
                      loss=criterion(output_img,target)
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()
                      #if input_transform % 50==0:
                        #%print("-------------Epoch ",epochs,"---------Loss:",loss.item(),"-------------Accuracy:", ((torch.sum(labels==torch.argmax(out, axis=1))/len(labels)).item())*100)
                      #sys#.stdout.flush()
                      
                      print(loss)
                      torch.cuda.empty_cache()
        # torch.save(self.model.state_dict(), '/content/mdoel/')
        




        


    def validate(self):
    #     model = Net()

        
        # Validation loop begin
        model = self.model

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model=model.to(device)

        # model.eval()

        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0)  #Momentum=0.9
        # scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9, last_epoch=-1, verbose=True)    
        star1 = os.path.join('/content/images/val', "*.jpg")
        star1 = glob.glob(star1)
        # print(len(star1)) #3862
        val_dataset = ColorizeData(star1)
        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

        

        for epoch in range(self.epochs):
          model.eval()
          # print("model enter")
          for input_transform,target_transform in val_dataloader:
            inputs = torch.tensor(input_transform).to(device)
            target = torch.tensor(target_transform).to(device)
            inputs,target = inputs.cuda(),target.cuda()
            
            output_img = model(inputs)
            # print("validation final output check")
            print(output_img.shape)
            loss=criterion(output_img,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        



        torch.save(self.model.state_dict(), '/content/model/model.pth')

        # img = Image.open('/content/images/val/1025.jpg')
        
        # transform = T.Compose([T.ToTensor(),
        #                                   T.Resize(size=(256,256)),
        #                                   T.Grayscale(),
        #                                   T.Normalize((0.5), (0.5))
        #                                   ]) 
        # # img = Image.open('/content/images/val/1025.jpg')  # Load image as PIL.Image
        # x = transform(img)  # Preprocess image
        # x = x.unsqueeze(0)  # Add batch dimension
        # print(x.shape)
        # # output = model(x)
        # final = self.model(img)
        # save_image(final,'ch.png')
    
        # checkoutput = self.model('/content/images/val/1025.jpg')
        # img = cv2.imread(checkoutput)
        # cv2_imshow(img)
    
        # self.model.load_state_dict(torch.load(filepath))
        # model.eval()
                # print("-------------Epoch ",epoch,"---------Loss:",loss.item(),"-------------Accuracy:", ((torch.sum(target==torch.argmax(out, axis=1))/len(target)).item())*100)
            # sys.stdout.flush()
            # print(loss)
            # torch.cuda.empty_cache()
            # print('Finished validation.')
        # return loss.avg
        # ------
        # Validation loop end
        # ------
        # Determine your evaluation metrics on the validation dataset.

In [7]:


def main():
  foo = Trainer()
  foo.train()
  foo.validate()

if __name__=='__main__':
  main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(0.5519, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3984, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1997, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1820, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1307, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1273, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1262, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1135, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1255, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1114, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0900, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1148, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0915, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0986, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0942, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1115, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0933, device='cuda:0', grad_fn=

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([

In [33]:
import matplotlib
from torchvision.utils import save_image
model = Net()  # Initialize model
model.load_state_dict(torch.load('/content/model/model.pth'))  # Load pretrained parameters
model.eval()  # Set to eval mode to change behavior of Dropout, BatchNorm

transform = T.Compose(    [T.ToTensor(),
                                          T.Resize(size=(256,256)),
                                          T.Grayscale(),
                                          T.Normalize((0.5), (0.5))
                                          ]) # Same as for your validation data, e.g. Resize, ToTensor, Normalize, ...

img = Image.open('/content/black-white-lotus-flower-isolated-260nw-1816183571.jpg')  # Load image as PIL.Image
x = transform(img)  # Preprocess image
x = x.unsqueeze(0)  # Add batch dimension

output = model(x)  # Forward pass

output = output.squeeze(0)
print(output.shape)
print(output)
xx = output.detach()
save_image(xx,'ch.png') #############SAVED IMAGE COLORED

torch.Size([3, 256, 256])
tensor([[[0.7327, 0.7327, 0.8248,  ..., 0.9183, 0.6088, 0.6088],
         [0.7327, 0.7327, 0.8248,  ..., 0.9183, 0.6088, 0.6088],
         [0.8452, 0.8452, 0.9715,  ..., 0.9276, 0.8833, 0.8833],
         ...,
         [0.2074, 0.2074, 0.6260,  ..., 0.8099, 0.8806, 0.8806],
         [0.2986, 0.2986, 0.6368,  ..., 0.6983, 0.6787, 0.6787],
         [0.2986, 0.2986, 0.6368,  ..., 0.6983, 0.6787, 0.6787]],

        [[0.7264, 0.7264, 0.8483,  ..., 0.9205, 0.6305, 0.6305],
         [0.7264, 0.7264, 0.8483,  ..., 0.9205, 0.6305, 0.6305],
         [0.8562, 0.8562, 0.9788,  ..., 0.9654, 0.8865, 0.8865],
         ...,
         [0.2800, 0.2800, 0.7069,  ..., 0.9870, 0.9636, 0.9636],
         [0.3394, 0.3394, 0.6999,  ..., 0.8063, 0.7875, 0.7875],
         [0.3394, 0.3394, 0.6999,  ..., 0.8063, 0.7875, 0.7875]],

        [[0.6954, 0.6954, 0.8238,  ..., 0.8175, 0.5495, 0.5495],
         [0.6954, 0.6954, 0.8238,  ..., 0.8175, 0.5495, 0.5495],
         [0.7959, 0.7959, 0.9215